In [1]:
import pandas as pd
import numpy as np

from cryptodatapy.extract.datarequest import DataRequest
from cryptodatapy.extract.getdata import GetData
from cryptodatapy.transform.od import OutlierDetection
from cryptodatapy.transform.impute import Impute
from cryptodatapy.transform.filter import Filter
from cryptodatapy.transform.clean import CleanData, stitch_dataframes
from cryptodatapy.transform.impute import Impute

fatal: bad revision 'HEAD'
Importing plotly failed. Interactive plots will not work.


In [2]:
# get all Binance perp futures tickers
data_req = DataRequest(source='ccxt')
perp_tickers = GetData(data_req).get_meta(method='get_markets_info', exch='binanceusdm', as_list=True)

In [3]:
len(perp_tickers)

314

In [4]:
'BTCST/USDT:USDT' in perp_tickers

True

In [5]:
# get Binance spot tickers
data_req = DataRequest(source='ccxt')
spot_tickers = GetData(data_req).get_meta(method='get_markets_info', exch='binance', as_list=True)

In [6]:
'BTCST/USDT:USDT' in spot_tickers

True

In [7]:
# find intersecting tickers
binance_tickers = [ticker for ticker in perp_tickers if ticker in spot_tickers]

In [8]:
# number of tickers
len(binance_tickers)

314

In [9]:
'BTCST/USDT:USDT' in binance_tickers

True

In [10]:
## # get cryptocompare tickers
data_req = DataRequest(source='cryptocompare')
cc_tickers = GetData(data_req).get_meta(method='get_assets_info', as_list=True)

In [11]:
'BTCST' in cc_tickers

True

In [12]:
# keep only USDT ticker
bin_tickers = []
for ticker in binance_tickers:
    if '/' in ticker and ticker.split('/')[1] == 'USDT:USDT':
        bin_tickers.append(ticker.split('/')[0])

In [13]:
'BTCST' in bin_tickers

True

In [14]:
# usdt tickers
usdt_tickers = [ticker.split('/')[0] for ticker in binance_tickers if '/'in ticker and ticker.split('/')[1] == 'USDT:USDT']

In [15]:
len(usdt_tickers)

282

In [16]:
# intersecting tickers
tickers = [ticker for ticker in usdt_tickers if ticker in cc_tickers]

In [17]:
len(tickers)

270

In [18]:
'BTCST' in tickers

True

### Binance Perp Futures

In [20]:
# pull daily OHLC and funding rates for perp futures on Binance USDM exchange
data_req = DataRequest(source='ccxt',
                       tickers=tickers, 
                       fields=['open', 'high', 'low', 'close', 'volume', 'funding_rate'], 
                       mkt_type='perpetual_future', 
                       freq='d')

In [21]:
df1 = GetData(data_req).get_series()

In [22]:
df1

open      high       low     close       volume  \
date       ticker                                                        
2019-09-08 BTC      10000.0  10412.65   10000.0  10391.63     3096.291   
2019-09-09 BTC     10316.62  10475.54  10077.22   10307.0    14824.373   
2019-09-10 BTC      10307.0  10382.97   9940.87  10102.02     9068.955   
2019-09-11 BTC     10094.27  10293.11   9884.31  10159.55    10897.922   
2019-09-12 BTC     10163.06  10450.13  10042.12  10415.13    15609.634   
...                     ...       ...       ...       ...          ...   
2024-08-13 ZETA      0.6558    0.7099    0.6143    0.6556  290931468.0   
           ZIL      0.01394   0.01397   0.01348   0.01372  211016383.0   
           ZK       0.11683   0.11895   0.11223   0.11713  180118593.0   
           ZRO        3.509     3.533     3.349     3.459   10802271.5   
           ZRX       0.3102    0.3126       0.3    0.3083   18072404.9   

                   funding_rate  
date       ticker                
2019-09-08 BTC             <NA>  
2019-09-09 BTC             <NA>  
2019-09-10 BTC           0.0002  
2019-09-11 BTC           0.0003  
2019-09-12 BTC           0.0003  
...                         ...  
2024-08-13 ZETA       -0.001465  
           ZIL           0.0002  
           ZK            0.0002  
           ZRO         0.000173  
           ZRX        -0.000044  

[222221 rows x 6 columns]

In [23]:
# df1.to_csv('binance_perp_futures.csv')
df1 = pd.read_csv('../../../../factorlab/notebooks/binance_perp_futures.csv', index_col=['date', 'ticker'], parse_dates=['date'])

In [24]:
df1.head()

,,open,high,low,close,volume,funding_rate
date,ticker,,,,,,
2019-09-08,BTC,10000.00,10412.65,10000.00,10391.63,3096.291,NaN
2019-09-09,BTC,10316.62,10475.54,10077.22,10307.00,14824.373,NaN
2019-09-10,BTC,10307.00,10382.97,9940.87,10102.02,9068.955,0.0002
2019-09-11,BTC,10094.27,10293.11,9884.31,10159.55,10897.922,0.0003
2019-09-12,BTC,10163.06,10450.13,10042.12,10415.13,15609.634,0.0003


### Binance Spot

In [25]:
# pull OHLC from Binance
data_req = DataRequest(source='ccxt',
                       tickers=tickers, 
                       fields=['open', 'high', 'low', 'close', 'volume'], 
                       freq='d')

In [26]:
df2 = GetData(data_req).get_series()

In [27]:
# df2.to_csv('binance_spot.csv')
df2 = pd.read_csv('../../../../factorlab/notebooks/binance_spot.csv', index_col=['date', 'ticker'], parse_dates=['date'])

In [28]:
df2.head()

open     high      low    close       volume
date       ticker                                                 
2017-08-17 BTC     4261.48  4485.39  4200.74  4285.08   795.150377
           ETH      301.13   312.18   298.00   302.00  7030.710340
2017-08-18 BTC     4285.08  4371.52  3938.77  4108.37  1199.888264
           ETH      302.00   311.79   283.94   293.96  9537.846460
2017-08-19 BTC     4108.37  4184.69  3850.00  4139.98   381.309763

### CryptoCompare - Historical Prices

In [29]:
# pull OHLC from CryptoCompare
data_req = DataRequest(source='cryptocompare',
                       tickers=tickers, 
                       fields=['open', 'high', 'low', 'close', 'volume'], 
                       freq='d')

In [30]:
df3 = GetData(data_req).get_series()

In [31]:
# df3.to_csv('cc_spot.csv')
df3 = pd.read_csv('../../../../factorlab/notebooks/cc_spot.csv', index_col=['date', 'ticker'], parse_dates=['date'])

### Clean Data

In [32]:
df = stitch_dataframes([df1, df2, df3])
df.funding_rate = df.funding_rate.fillna(0)

In [33]:
df.head()

,,open,high,low,close,volume,funding_rate
date,ticker,,,,,,
2010-07-17,BTC,0.04951,0.04951,0.04951,0.04951,20.00,0.0
2010-07-18,BTC,0.04951,0.08585,0.04951,0.08584,75.01,0.0
2010-07-19,BTC,0.08584,0.09307,0.07723,0.08080,574.00,0.0
2010-07-20,BTC,0.08080,0.08181,0.07426,0.07474,262.00,0.0
2010-07-21,BTC,0.07474,0.07921,0.06634,0.07921,575.00,0.0


In [34]:
df.index.get_level_values(1).unique()

Index(['BTC', 'LTC', 'DOGE', 'DASH', 'XLM', 'XMR', 'XRP', 'KEY', 'DGB', 'XEM',
       ...
       'TNSR', 'SAGA', 'REZ', 'BB', 'NOT', 'IO', 'ZK', 'LISTA', 'ZRO',
       'RENDER'],
      dtype='object', name='ticker', length=256)

In [ ]:
delisted_tickers = ['AGIX', 'CTK', 'CVC', 'CVX', 'DGB', 'FTT', 'GLMR', 'IDEX', 'MDT',
       'OCEAN', 'RAD', 'RAY', 'SC', 'SLP', 'SNT', 'STPT', 'STRAX', 'WAVES']

In [35]:
clean = CleanData(df)

In [36]:
clean.filter_delisted_tickers()

In [37]:
clean.filtered_tickers

['CVC',
 'CTK',
 'STRAX',
 'STPT',
 'DGB',
 'CVX',
 'FTT',
 'GLMR',
 'OCEAN',
 'SLP',
 'AGIX',
 'RAY',
 'MDT',
 'SC',
 'SNT',
 'IDEX',
 'RAD',
 'WAVES']

In [38]:
clean.filter_outliers(od_method='mad', excl_cols=['volume', 'funding_rate'], thresh_val=10)

In [39]:
clean.df

open     high      low    close
date       ticker                                    
2010-07-17 BTC        <NA>     <NA>     <NA>     <NA>
2010-07-18 BTC        <NA>     <NA>     <NA>     <NA>
2010-07-19 BTC        <NA>     <NA>     <NA>     <NA>
2010-07-20 BTC        <NA>     <NA>     <NA>     <NA>
2010-07-21 BTC        <NA>     <NA>     <NA>     <NA>
...                    ...      ...      ...      ...
2024-08-04 ZEN        <NA>    9.112    8.285    8.462
           ZIL     0.01422  0.01441  0.01392  0.01396
           ZK      0.11451  0.11673  0.10857  0.10874
           ZRO       3.631    3.648    3.362    3.367
           ZRX      0.3055   0.3121   0.2983   0.2988

[357696 rows x 4 columns]

In [40]:
clean.repair_outliers(imp_method='fcst')

In [41]:
clean.df

open     high      low    close        volume  \
date       ticker                                                     
2010-07-17 BTC       42.99    43.76    40.99    41.01  2.000000e+01   
2010-07-18 BTC     0.07921  0.08181  0.06634  0.07921  7.501000e+01   
2010-07-19 BTC     0.07474  0.07921   0.0505  0.06262  5.740000e+02   
2010-07-20 BTC     0.06868  0.07344   0.0505  0.06052  2.620000e+02   
2010-07-21 BTC     0.06262  0.06767   0.0505  0.05842  5.750000e+02   
...                    ...      ...      ...      ...           ...   
2024-08-04 ZEN       9.657    9.112    8.285    8.462  2.071124e+06   
           ZIL     0.01422  0.01441  0.01392  0.01396  2.048626e+08   
           ZK      0.11451  0.11673  0.10857  0.10874  3.833253e+08   
           ZRO       3.631    3.648    3.362    3.367  7.049472e+07   
           ZRX      0.3055   0.3121   0.2983   0.2988  9.810764e+06   

                   funding_rate  
date       ticker                
2010-07-17 BTC         0.000000  
2010-07-18 BTC         0.000000  
2010-07-19 BTC         0.000000  
2010-07-20 BTC         0.000000  
2010-07-21 BTC         0.000000  
...                         ...  
2024-08-04 ZEN         0.000194  
           ZIL        -0.000031  
           ZK          0.000200  
           ZRO         0.000184  
           ZRX         0.000186  

[357696 rows x 6 columns]

In [42]:
clean.filter_avg_trading_val(thresh_val=1000000)

In [43]:
clean.df

open     high      low    close       volume  \
date       ticker                                                    
2010-07-17 BTC        <NA>     <NA>     <NA>     <NA>          NaN   
2010-07-18 BTC        <NA>     <NA>     <NA>     <NA>          NaN   
2010-07-19 BTC        <NA>     <NA>     <NA>     <NA>          NaN   
2010-07-20 BTC        <NA>     <NA>     <NA>     <NA>          NaN   
2010-07-21 BTC        <NA>     <NA>     <NA>     <NA>          NaN   
...                    ...      ...      ...      ...          ...   
2024-08-04 ZEN       9.657    9.112    8.285    8.462    2071124.0   
           ZIL     0.01422  0.01441  0.01392  0.01396  204862627.0   
           ZK      0.11451  0.11673  0.10857  0.10874  383325323.0   
           ZRO       3.631    3.648    3.362    3.367   70494717.0   
           ZRX      0.3055   0.3121   0.2983   0.2988    9810764.1   

                   funding_rate  
date       ticker                
2010-07-17 BTC              NaN  
2010-07-18 BTC              NaN  
2010-07-19 BTC              NaN  
2010-07-20 BTC              NaN  
2010-07-21 BTC              NaN  
...                         ...  
2024-08-04 ZEN         0.000194  
           ZIL        -0.000031  
           ZK          0.000200  
           ZRO         0.000184  
           ZRX         0.000186  

[357696 rows x 6 columns]

In [44]:
clean.filtered_tickers

['CVC',
 'CTK',
 'STRAX',
 'STPT',
 'DGB',
 'CVX',
 'FTT',
 'GLMR',
 'OCEAN',
 'SLP',
 'AGIX',
 'RAY',
 'MDT',
 'SC',
 'SNT',
 'IDEX',
 'RAD',
 'WAVES']

In [45]:
clean.filter_missing_vals_gaps()

In [47]:
clean.filtered_tickers

['CVC',
 'CTK',
 'STRAX',
 'STPT',
 'DGB',
 'CVX',
 'FTT',
 'GLMR',
 'OCEAN',
 'SLP',
 'AGIX',
 'RAY',
 'MDT',
 'SC',
 'SNT',
 'IDEX',
 'RAD',
 'WAVES']

In [48]:
clean.filter_min_nobs(ts_obs=1400, cs_obs=5)

In [49]:
clean.df

open     high      low    close        volume  \
date       ticker                                                     
2017-06-18 BAT        <NA>     <NA>     <NA>     <NA>           NaN   
           BTC      2655.1  2676.04  2488.59  2539.56  9.200422e+04   
           DASH       <NA>     <NA>     <NA>     <NA>           NaN   
           DOGE       <NA>     <NA>     <NA>     <NA>           NaN   
           ETC       21.98     23.8     19.5    20.27  1.306320e+06   
...                    ...      ...      ...      ...           ...   
2024-08-04 XTZ       0.685    0.693    0.677    0.679  5.373374e+06   
           YFI      5341.0   5341.0   5196.0   5198.0  4.281050e+02   
           ZEC       31.76    34.44    31.28    31.55  1.348085e+06   
           ZIL     0.01422  0.01441  0.01392  0.01396  2.048626e+08   
           ZRX      0.3055   0.3121   0.2983   0.2988  9.810764e+06   

                   funding_rate  
date       ticker                
2017-06-18 BAT              NaN  
           BTC         0.000000  
           DASH             NaN  
           DOGE             NaN  
           ETC         0.000000  
...                         ...  
2024-08-04 XTZ         0.000200  
           YFI         0.000169  
           ZEC         0.000191  
           ZIL        -0.000031  
           ZRX         0.000186  

[136630 rows x 6 columns]

In [50]:
# # clean data
# clean_df = clean.filter_delisted_tickers().\
#                          filter_outliers(od_method='mad', excl_cols=['volume', 'funding_rate'], thresh_val=10).\
#                          repair_outliers(imp_method='fcst').\
#                          filter_avg_trading_val(thresh_val=1000000).\
#                          filter_missing_vals_gaps().\
#                          filter_min_nobs(ts_obs=1500, cs_obs=10).\
#                          get(attr='df').dropna(how='all')

In [53]:
df.to_parquet('s3://factorlab-data/binance_historical_ohlcv_daily.parquet')

In [54]:
clean.df.dropna(how='all').to_parquet('../../../../factorlab/notebooks/binance_historical_ohlcv_daily.parquet')

In [55]:
clean.df.dropna(how='all').to_csv('../../../../factorlab/notebooks/binance_historical_ohlcv_daily.csv')